In [4]:
####################################Imports and Initial Feed Rip###################################################
import feedparser as fp, pandas as pd, datetime, time, string, requests as r
from bs4 import BeautifulSoup 
from functools import reduce


rss_feed = 'http://rss.nytimes.com/services/xml/rss/nyt/Technology.xml'
company_info_df = pd.read_csv('Target_Company.csv')
company_name = company_info_df.iloc[0]['Company Name']


#Get the News Feed and Slice it to only include the news items
feed = fp.parse(rss_feed)
feed_items = feed['items']
print(feed_items[0])
###################################################################################################################

{'author': 'MIKE ISAAC and MICHAEL J. de la MERCED', 'tags': [{'label': None, 'term': 'Football', 'scheme': 'http://www.nytimes.com/namespaces/keywords/des'}, {'label': None, 'term': 'Video Recordings, Downloads and Streaming', 'scheme': 'http://www.nytimes.com/namespaces/keywords/des'}, {'label': None, 'term': 'Twitter', 'scheme': 'http://www.nytimes.com/namespaces/keywords/nyt_org_all'}, {'label': None, 'term': 'National Football League', 'scheme': 'http://www.nytimes.com/namespaces/keywords/nyt_org_all'}, {'label': None, 'term': 'Reddit Inc', 'scheme': 'http://www.nytimes.com/namespaces/keywords/nyt_org_all'}, {'label': None, 'term': 'Facebook Inc', 'scheme': 'http://www.nytimes.com/namespaces/keywords/nyt_org_all'}], 'author_detail': {'name': 'MIKE ISAAC and MICHAEL J. de la MERCED'}, 'media_credit': [{'content': 'Joshua Bright for The New York Times', 'scheme': 'urn:ebu'}], 'title': 'Bits: The Week in Tech: Facebook Live, a More Civil Reddit and Yahoo’s Odd Deal-Making', 'authors'

In [5]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','summary']])
    
    for i in range(0,len(being_cleaned_df['summary'])):
        soup = BeautifulSoup(being_cleaned_df['summary'][i],'lxml')
        being_cleaned_df['summary'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the Full Dataframe    
full_df = dataframe_cleaner(raw_news_dataframe)
,#Create the filtered dataframe, containing only rows that have company_name in the title
company_in_title_list = list(map(lambda x: company_name in x,full_df['title']))
news_df = full_df[company_in_title_list].reset_index(drop=True)
news_df.head()

###################################################################################################################

,author,id,title,summary
0,JAMIL N. JAFFER and DANIEL J. ROSENTHAL,http://www.nytimes.com/2016/04/09/opinion/why-...,Op-Ed Contributors: Why Apple’s Stand Against ...,Privacy advocates applauded the decision not t...
1,ERIC LICHTBLAU and KATIE BENNER,http://www.nytimes.com/2016/04/09/technology/u...,U.S. Presses Bid to Force Apple to Unlock iPho...,"In a letter to a federal judge, the Justice De..."


In [6]:
#Load in the Sentiment DataFrame
sentiment_df = pd.read_csv('../Sentiment_Dictionary/sentiment_DataFrame.csv')
sentiment_df = sentiment_df.ix[:,'type':]
sentiment_df.head(1)

,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1


In [7]:
##########################Perform Sentiment Analysis on the Title and Body of the Articles#########################
def title_analysis(title):
    tokenized_title = dict.fromkeys(title.split(' '))                          
                                
    for key in tokenized_title:
        tokenized_title[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_title.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_title)
    
    return normalized_sentiment

def summary_analysis(summary):
    tokenized_summary = dict.fromkeys(summary.split(' '))
    
    for key in tokenized_summary:
        tokenized_summary[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_summary.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_summary)
    return normalized_sentiment

def sentiment_analysis(dataframe):
    analyzing_df = dataframe
    analyzing_df['normalized_title_sentiment'] = list(map(lambda x: title_analysis(x),analyzing_df['title']))
    analyzing_df['normalized_summary_sentiment'] = list(map(lambda x: summary_analysis(x),analyzing_df['summary']))
    return analyzing_df

start_time = time.time()
analyzed_articles_df = sentiment_analysis(news_df)
elapsed_time = time.time() - start_time
print('Elapsed Time: ' + str(elapsed_time))
analyzed_articles_df.head(1)
analyzed_articles_df.to_csv('NYTSentiment.csv')
###################################################################################################################

Elapsed Time: 3.7850706577301025
